In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [12]:
Corpus = pd.read_csv("train_data.csv")

In [13]:
Corpus.drop(["Unnamed: 0","Unnamed: 0.1"],axis=1,inplace=True)
Corpus.drop(["1","2"],axis=1,inplace=True)

In [14]:
Corpus.columns=["text","label"]

In [15]:
Corpus.head()

,text,label
0,TextVectorization inconsistency depending on o...,bug
1,Cant install tflite_runtime while deploying on...,build
2,[TF2.3.0] Failed to Build libtensorflow_cc.so ...,build
3,Optimize unique_op by repetition rate,feature
4,pip and conda install tensorflow problems in a...,build


In [17]:
print(type(Corpus["text"]))

<class 'pandas.core.series.Series'>


In [18]:
Corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33062 entries, 0 to 33061
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    33061 non-null  object
 1   label   33062 non-null  object
dtypes: object(2)
memory usage: 516.7+ KB


In [21]:
Corpus=Corpus.astype({"text": str})

In [28]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/varun/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /home/varun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/varun/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [29]:
Corpus['text'].dropna(inplace=True)
Corpus=Corpus.astype({"text": str})
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Corpus['text'] = [entry.lower() for entry in Corpus['text']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
Corpus['text']= [word_tokenize(entry) for entry in Corpus['text']]
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus.loc[index,'text_final'] = str(Final_words)